# Experiments on parameters

This notebook can be used to experiment with the different parameters of the models and see how those affect the outcome of the training. The discussion of the results for each hyper-parameter will be divided between the autoencoder and the deep tica models. As the hyper-parameters might affect them differently.

Unlike with alanine dipeptide, here we don't have access to a lower dimensional manifold that separates the metastable states. Thus we cannot simply project the CV onto the $\phi$, $\psi$ space and see how well the model is able to separate the metastable states. Instead, knowing that each trajectory shows a transition between two states, we can project the trajectory on the CV space and see how smoothly it separates the initial and the final states.

In [1]:
from deep_cartograph.run import deep_cartograph
import importlib.resources as resources
from deep_cartograph import data

from IPython.display import display, HTML
from typing import Literal, Dict
from decimal import Decimal
import logging
import shutil
import yaml
import time
import os

# Get the path to the data
data_folder = resources.files(data)

# Set logging level
logging.basicConfig(level=logging.INFO)
    
def run_deep_cartograph(configuration: Dict, features: Literal['torsions', 'distances'], system_name: str, test_name: str):
    """
    Runs deep cartograph using one of the system trajectories. Using one of the feature dictionaries
    and the provided configuration.

    Input
    -----

        configuration: configuration for all the steps of the deep cartograph main workflow
        features:      choice of features to use to train/compute the CVs
        system_name:   name of the system to use
        test_name:     name of the test that will be used as output folder name
    """
    # Input trajectory and topology
    input_path = f"{data_folder}/calpha_transitions/input/{system_name}"
    traj_path = os.path.join(input_path, f'{system_name}.dcd')
    top_path = os.path.join(input_path, f'{system_name}.pdb')
        
    # Output folder
    output_folder = f"{data_folder}/calpha_transitions/experiments/output/{system_name}/{features}/{test_name}"

    # Clean output folder
    if os.path.exists(output_folder):
        shutil.rmtree(output_folder)

    # Run workflow 
    deep_cartograph(
        configuration = configuration,
        trajectory = traj_path,
        topology = top_path,
        output_folder = output_folder)

def show_results(features: Literal['torsions', 'distances'], system_name: str, model_name: str, test_name: str):
    """
    Show the results for a specific model trained with deep cartograph

    Inputs
    ------

        features:    choice of features to use to train/compute the CVs
        system_name: name of the system
        model_name:  name of the model
        test_name:   name of the test to show results for
    """
    def show_score(score_path):
        """
        Print score path in a nice format 

        Input
        -----

            score_path: path to the score file
        """

        # Read score
        with open(score_path, 'r') as file:
            score = file.read()

        # Print score in scientific notation
        print(f"Final model score: {Decimal(score):.4E}")

    def show_eigenvalues(eig_path):
        """
        Print eigenvalues in a nice format

        Input
        -----

            eig_path: path to the eigenvalues file
        """

        # Read eigenvalues
        with open(eig_path, 'r') as file:
            eigenvalues = file.readlines()

        # Print eigenvalues
        for i, eig in enumerate(eigenvalues):
            print(f"Eigenvalue {i+1}: {Decimal(eig):.4E}")

    # Output folder
    output_folder = f"{data_folder}/calpha_transitions/experiments/output/{system_name}/{features}/{test_name}"

    # Show output folder
    print(f"Output folder: {output_folder}")

    # Find model path
    model_path = os.path.join(output_folder, "train_colvars", model_name)

    # Find score if any
    if model_name in ['ae', 'deep_tica']:
        score_path = os.path.join(model_path, "model_score.txt")
        if os.path.exists(score_path):
            show_score(score_path)
        else:
            print("Score file not found")

    # Find eigenvalues if any
    if model_name == 'deep_tica':
        eig_path = os.path.join(model_path, "eigenvalues.txt")
        if os.path.exists(eig_path):
            show_eigenvalues(eig_path)
        else:
            print("Eigenvalues file not found")

    # Paths to images
    loss = os.path.join(model_path, 'loss.png')
    eigenvalues = os.path.join(model_path, 'eigenvalues.png')
    trajectory = os.path.join(model_path, 'trajectory.png')
    paths = [loss, eigenvalues, trajectory]

    # Generate HTML image tags
    timestamp = int(time.time()) # Add timestamp to avoid caching
    images_html = [f'<img src="{path}?{timestamp}" style="width: 600px; margin-right: 10px;">' for path in paths if os.path.exists(path)]

    # Display images
    display(HTML(''.join(images_html)))

## Batch size

Smaller batch sizes can introduce more noise in the gradient updates, which can help the model scape local or sharp minima and find better solutions. Thus generalize better (see relation between training and validation loss). However, smaller batch sizes can also slow down the training process. In this case we have a small dataset and model, so the recommendation is to use a small batch size.

In [5]:

features = 'torsions'

# Load initial configuration
config_path = f"{data_folder}/calpha_transitions/experiments/{features}_config.yml"

with open(config_path, 'r') as file:
    configuration = yaml.load(file, Loader = yaml.FullLoader)
    
batch_sizes = [16, 32, 64, 128, 256]   # 256 is close to all the samples in the training set (330*0.8 = 264)
system_name = '2olu_A-2olv_A-1'        # Has around 330 frames

for bs in batch_sizes:

    # Test name
    test_name = f"bs_{bs}"

    # Make a copy of the initial configuration
    test_configuration = configuration.copy()

    # Update batch size
    test_configuration['train_colvars']['common']['training']['general']['batch_size'] = bs

    # Run deep cartograph
    run_deep_cartograph(test_configuration, features, system_name, test_name)

INFO:deep_cartograph:================
INFO:deep_cartograph:Compute features
INFO:deep_cartograph:================
INFO:deep_cartograph:Trajectory file: ../../../data/calpha_transitions/input/2olu_A-2olv_A-1/2olu_A-2olv_A-1.dcd
/shared/work/pnavarro/envs/deep_cartograph/lib/python3.10/site-packages/MDAnalysis/topology/PDBParser.py:317: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn("Element information is missing, elements attribute "
/shared/work/pnavarro/envs/deep_cartograph/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:747: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/shared/work/pnavarro/envs/deep_cartograph/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1078: UserWarning: Found no information for attr: 'elements' Using default value of ' '
  warning

<Figure size 500x400 with 0 Axes>

<Figure size 500x400 with 0 Axes>

<Figure size 500x400 with 0 Axes>

<Figure size 500x400 with 0 Axes>

...

<Figure size 500x400 with 0 Axes>

<Figure size 500x400 with 0 Axes>

<Figure size 500x400 with 0 Axes>

<Figure size 500x400 with 0 Axes>

<Figure size 500x400 with 0 Axes>

In [6]:
# Show results for the Autoencoder

for bs in batch_sizes:

    print(f"Batch size: {bs} ----------------------------------------------- #")

    # Test name
    test_name = f"bs_{bs}"
    
    # Show main results
    show_results(features, system_name, 'ae', test_name)

Batch size: 16 ----------------------------------------------- #
Output folder: ../../../data/calpha_transitions/experiments/output/2olu_A-2olv_A-1/torsions/bs_16
Final model score: 2.8088E-3


Batch size: 32 ----------------------------------------------- #
Output folder: ../../../data/calpha_transitions/experiments/output/2olu_A-2olv_A-1/torsions/bs_32
Final model score: 2.8263E-3


Batch size: 64 ----------------------------------------------- #
Output folder: ../../../data/calpha_transitions/experiments/output/2olu_A-2olv_A-1/torsions/bs_64
Final model score: 3.0075E-3


Batch size: 128 ----------------------------------------------- #
Output folder: ../../../data/calpha_transitions/experiments/output/2olu_A-2olv_A-1/torsions/bs_128
Final model score: 3.2034E-3


Batch size: 256 ----------------------------------------------- #
Output folder: ../../../data/calpha_transitions/experiments/output/2olu_A-2olv_A-1/torsions/bs_256
Final model score: 4.3471E-3


- We can see that as the batch size increases, the validation loss takes more epochs to decrease
- Also, the training loss follows more closely the validation loss

In [7]:
# Show results for Deep TICA

for bs in batch_sizes:

    print(f"Batch size: {bs} ----------------------------------------------- #")

    # Test name
    test_name = f"bs_{bs}"
    
    # Show main results
    show_results(features, system_name, 'deep_tica', test_name)

Batch size: 16 ----------------------------------------------- #
Output folder: ../../../data/calpha_transitions/experiments/output/2olu_A-2olv_A-1/torsions/bs_16
Final model score: -2.5727E+3
Eigenvalue 1: 1.1717E+1
Eigenvalue 2: -2.0833E-1


Batch size: 32 ----------------------------------------------- #
Output folder: ../../../data/calpha_transitions/experiments/output/2olu_A-2olv_A-1/torsions/bs_32
Final model score: -1.8339E+0
Eigenvalue 1: 9.7983E-1
Eigenvalue 2: 9.3382E-1


Batch size: 64 ----------------------------------------------- #
Output folder: ../../../data/calpha_transitions/experiments/output/2olu_A-2olv_A-1/torsions/bs_64
Final model score: -1.9195E+0
Eigenvalue 1: 9.8996E-1
Eigenvalue 2: 9.6897E-1


Batch size: 128 ----------------------------------------------- #
Output folder: ../../../data/calpha_transitions/experiments/output/2olu_A-2olv_A-1/torsions/bs_128
Final model score: -1.8908E+0
Eigenvalue 1: 9.8483E-1
Eigenvalue 2: 9.5964E-1


Batch size: 256 ----------------------------------------------- #
Output folder: ../../../data/calpha_transitions/experiments/output/2olu_A-2olv_A-1/torsions/bs_256
Final model score: -1.8662E+0
Eigenvalue 1: 9.9245E-1
Eigenvalue 2: 9.3873E-1


## Learning rate

It controls the size of the steps that the optimizer takes in the direction of the gradient. If the learning rate is too high, the optimizer might overshoot the minimum and diverge. If the learning rate is too low, the optimizer might take too long to converge.

In [9]:
features = 'torsions'

# Load initial configuration
config_path = f"{data_folder}/calpha_transitions/experiments/{features}_config.yml"

with open(config_path, 'r') as file:
    configuration = yaml.load(file, Loader = yaml.FullLoader)
    
learning_rates = [1e-2, 1e-3, 1e-4, 1e-5]
system_name = '2olu_A-2olv_A-1'        # Has around 330 frames

for lr in learning_rates:

    # Test name
    test_name = f"lr_{lr}"

    # Make a copy of the initial configuration
    test_configuration = configuration.copy()

    # Update learning rate
    test_configuration['train_colvars']['common']['training']['optimizer']['kwargs']['lr'] = lr

    # Run deep cartograph
    run_deep_cartograph(test_configuration, features, system_name, test_name)

INFO:deep_cartograph:================
INFO:deep_cartograph:Compute features
INFO:deep_cartograph:================
INFO:deep_cartograph:Trajectory file: ../../../data/calpha_transitions/input/2olu_A-2olv_A-1/2olu_A-2olv_A-1.dcd
/shared/work/pnavarro/envs/deep_cartograph/lib/python3.10/site-packages/MDAnalysis/topology/PDBParser.py:317: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn("Element information is missing, elements attribute "
/shared/work/pnavarro/envs/deep_cartograph/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:747: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/shared/work/pnavarro/envs/deep_cartograph/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1078: UserWarning: Found no information for attr: 'elements' Using default value of ' '
  warning

<Figure size 500x400 with 0 Axes>

<Figure size 500x400 with 0 Axes>

<Figure size 500x400 with 0 Axes>

<Figure size 500x400 with 0 Axes>

...

<Figure size 500x400 with 0 Axes>

<Figure size 500x400 with 0 Axes>

<Figure size 500x400 with 0 Axes>

<Figure size 500x400 with 0 Axes>

<Figure size 500x400 with 0 Axes>

In [10]:
# Show results for the Autoencoder

for lr in learning_rates:

    print(f"Learning rate: {lr} ----------------------------------------------- #")

    # Test name
    test_name = f"lr_{lr}"
    
    # Show main results
    show_results(features, system_name, 'ae', test_name)

Learning rate: 0.01 ----------------------------------------------- #
Output folder: ../../../data/calpha_transitions/experiments/output/2olu_A-2olv_A-1/torsions/lr_0.01
Final model score: 2.9002E-3


Learning rate: 0.001 ----------------------------------------------- #
Output folder: ../../../data/calpha_transitions/experiments/output/2olu_A-2olv_A-1/torsions/lr_0.001
Final model score: 3.0075E-3


Learning rate: 0.0001 ----------------------------------------------- #
Output folder: ../../../data/calpha_transitions/experiments/output/2olu_A-2olv_A-1/torsions/lr_0.0001
Final model score: 3.9368E-3


Learning rate: 1e-05 ----------------------------------------------- #
Output folder: ../../../data/calpha_transitions/experiments/output/2olu_A-2olv_A-1/torsions/lr_1e-05
Final model score: 6.2269E-3


- From these results we can see that the Autoencoder needs a lower learning rate. Thus making it more expensive to train than Deep TICA.
- Note that a lower loss after training does not necessarily mean a better model (see trajectory unfolding on each CV space).

In [11]:
# Show results for the Deep TICA

for lr in learning_rates:

    print(f"Learning rate: {lr} ----------------------------------------------- #")

    # Test name
    test_name = f"lr_{lr}"
    
    # Show main results
    show_results(features, system_name, 'deep_tica', test_name)

Learning rate: 0.01 ----------------------------------------------- #
Output folder: ../../../data/calpha_transitions/experiments/output/2olu_A-2olv_A-1/torsions/lr_0.01
Final model score: -1.9674E+0
Eigenvalue 1: 9.9955E-1
Eigenvalue 2: 9.8246E-1


Learning rate: 0.001 ----------------------------------------------- #
Output folder: ../../../data/calpha_transitions/experiments/output/2olu_A-2olv_A-1/torsions/lr_0.001
Final model score: -1.9195E+0
Eigenvalue 1: 9.8996E-1
Eigenvalue 2: 9.6897E-1


Learning rate: 0.0001 ----------------------------------------------- #
Output folder: ../../../data/calpha_transitions/experiments/output/2olu_A-2olv_A-1/torsions/lr_0.0001
Final model score: -1.8395E+0
Eigenvalue 1: 9.9342E-1
Eigenvalue 2: 9.2213E-1


Learning rate: 1e-05 ----------------------------------------------- #
Output folder: ../../../data/calpha_transitions/experiments/output/2olu_A-2olv_A-1/torsions/lr_1e-05
Final model score: -1.8306E+0
Eigenvalue 1: 9.9089E-1
Eigenvalue 2: 9.2014E-1


- The Deep TICA model is more robust to the learning rate, and it can be trained with a higher learning rate. For the high learning rate, the model can be trained with a single epoch.

## Dropout

Dropout is a regularization technique that consists in randomly setting a fraction of the input tensor to zero during training. This helps the model to generalize better and avoid overfitting. The dropout rate is the fraction of the input tensor that is set to zero.

In [12]:
features = 'torsions'

# Load initial configuration
config_path = f"{data_folder}/calpha_transitions/experiments/{features}_config.yml"

with open(config_path, 'r') as file:
    configuration = yaml.load(file, Loader = yaml.FullLoader)
    
dropout_rates = [0.05, 0.1, 0.2, 0.3, 0.4]
system_name = '2olu_A-2olv_A-1'        # Has around 330 frames

for dr in dropout_rates:

    # Test name
    test_name = f"dr_{dr}"

    # Make a copy of the initial configuration
    test_configuration = configuration.copy()

    # Update learning rate
    test_configuration['train_colvars']['common']['training']['general']['dropout'] = dr

    # Run deep cartograph
    run_deep_cartograph(test_configuration, features, system_name, test_name)

INFO:deep_cartograph:================
INFO:deep_cartograph:Compute features
INFO:deep_cartograph:================
INFO:deep_cartograph:Trajectory file: ../../../data/calpha_transitions/input/2olu_A-2olv_A-1/2olu_A-2olv_A-1.dcd
/shared/work/pnavarro/envs/deep_cartograph/lib/python3.10/site-packages/MDAnalysis/topology/PDBParser.py:317: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn("Element information is missing, elements attribute "
/shared/work/pnavarro/envs/deep_cartograph/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:747: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/shared/work/pnavarro/envs/deep_cartograph/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1078: UserWarning: Found no information for attr: 'elements' Using default value of ' '
  warning

<Figure size 500x400 with 0 Axes>

<Figure size 500x400 with 0 Axes>

<Figure size 500x400 with 0 Axes>

<Figure size 500x400 with 0 Axes>

...

<Figure size 500x400 with 0 Axes>

<Figure size 500x400 with 0 Axes>

<Figure size 500x400 with 0 Axes>

<Figure size 500x400 with 0 Axes>

<Figure size 500x400 with 0 Axes>

In [15]:
# Show results for the Autoencoder

for dr in dropout_rates:

    print(f"Dropout rate: {dr} ----------------------------------------------- #")

    # Test name
    test_name = f"dr_{dr}"
    
    # Show main results
    show_results(features, system_name, 'ae', test_name)

Dropout rate: 0.05 ----------------------------------------------- #
Output folder: ../../../data/calpha_transitions/experiments/output/2olu_A-2olv_A-1/torsions/dr_0.05
Final model score: 2.4002E-3


Dropout rate: 0.1 ----------------------------------------------- #
Output folder: ../../../data/calpha_transitions/experiments/output/2olu_A-2olv_A-1/torsions/dr_0.1
Final model score: 3.0075E-3


Dropout rate: 0.2 ----------------------------------------------- #
Output folder: ../../../data/calpha_transitions/experiments/output/2olu_A-2olv_A-1/torsions/dr_0.2
Final model score: 4.3447E-3


Dropout rate: 0.3 ----------------------------------------------- #
Output folder: ../../../data/calpha_transitions/experiments/output/2olu_A-2olv_A-1/torsions/dr_0.3
Final model score: 5.0353E-3


Dropout rate: 0.4 ----------------------------------------------- #
Output folder: ../../../data/calpha_transitions/experiments/output/2olu_A-2olv_A-1/torsions/dr_0.4
Final model score: 6.9461E-3


In [14]:
# Show results for the Deep TICA

for dr in dropout_rates:

    print(f"Dropout rate: {dr} ----------------------------------------------- #")

    # Test name
    test_name = f"dr_{dr}"
    
    # Show main results
    show_results(features, system_name, 'deep_tica', test_name)

Dropout rate: 0.05 ----------------------------------------------- #
Output folder: ../../../data/calpha_transitions/experiments/output/2olu_A-2olv_A-1/torsions/dr_0.05
Final model score: -1.8999E+0
Eigenvalue 1: 9.9770E-1
Eigenvalue 2: 9.5054E-1


Dropout rate: 0.1 ----------------------------------------------- #
Output folder: ../../../data/calpha_transitions/experiments/output/2olu_A-2olv_A-1/torsions/dr_0.1
Final model score: -1.9195E+0
Eigenvalue 1: 9.8996E-1
Eigenvalue 2: 9.6897E-1


Dropout rate: 0.2 ----------------------------------------------- #
Output folder: ../../../data/calpha_transitions/experiments/output/2olu_A-2olv_A-1/torsions/dr_0.2
Final model score: -1.9188E+0
Eigenvalue 1: 9.9002E-1
Eigenvalue 2: 9.6874E-1


Dropout rate: 0.3 ----------------------------------------------- #
Output folder: ../../../data/calpha_transitions/experiments/output/2olu_A-2olv_A-1/torsions/dr_0.3
Final model score: -2.0693E+0
Eigenvalue 1: 1.1236E+0
Eigenvalue 2: 7.2557E-1


Dropout rate: 0.4 ----------------------------------------------- #
Output folder: ../../../data/calpha_transitions/experiments/output/2olu_A-2olv_A-1/torsions/dr_0.4
Final model score: -5.5411E+0
Eigenvalue 1: 1.3821E+0
Eigenvalue 2: 7.0388E-1


- The Autoncoder model benefits from a higher dropout rate. This might be because the Autoencoder has more parameters.

## Lag time

Lag time is the number of frames between pairs of input samples. It is used in the TICA and Deep TICA models.

In [13]:
features = 'torsions'

# Load initial configuration
config_path = f"{data_folder}/calpha_transitions/experiments/{features}_config.yml"

with open(config_path, 'r') as file:
    configuration = yaml.load(file, Loader = yaml.FullLoader)
    
lag_times = [1, 2, 4, 8, 16]
system_name = '2olu_A-2olv_A-1'        # Has around 330 frames

for lt in lag_times:

    # Test name
    test_name = f"lt_{lt}"

    # Make a copy of the initial configuration
    test_configuration = configuration.copy()

    # Update learning rate
    test_configuration['train_colvars']['common']['architecture']['lag_time'] = lt

    # Run deep cartograph
    run_deep_cartograph(test_configuration, features, system_name, test_name)

INFO:deep_cartograph:================
INFO:deep_cartograph:Compute features
INFO:deep_cartograph:================
INFO:deep_cartograph:Trajectory file: ../../../data/calpha_transitions/input/2olu_A-2olv_A-1/2olu_A-2olv_A-1.dcd
/shared/work/pnavarro/envs/deep_cartograph/lib/python3.10/site-packages/MDAnalysis/topology/PDBParser.py:317: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn("Element information is missing, elements attribute "
/shared/work/pnavarro/envs/deep_cartograph/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:747: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/shared/work/pnavarro/envs/deep_cartograph/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1078: UserWarning: Found no information for attr: 'elements' Using default value of ' '
  warning

[Warning] Normalization: the following features have a range of values < 1e-6: tensor([[1]])


INFO:deep_cartograph:Elapsed time (Train colvars): 00 h 00 min 21 s
INFO:deep_cartograph:Total elapsed time: 00 h 00 min 54 s
INFO:deep_cartograph:================
INFO:deep_cartograph:Compute features
INFO:deep_cartograph:================
INFO:deep_cartograph:Trajectory file: ../../../data/calpha_transitions/input/2olu_A-2olv_A-1/2olu_A-2olv_A-1.dcd
/shared/work/pnavarro/envs/deep_cartograph/lib/python3.10/site-packages/MDAnalysis/topology/PDBParser.py:317: UserWarning: Element information is missing, elements attribute will not be populated. If needed these can be guessed using MDAnalysis.topology.guessers.
  warnings.warn("Element information is missing, elements attribute "
/shared/work/pnavarro/envs/deep_cartograph/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:747: UserWarning: Unit cell dimensions not found. CRYST1 record set to unitary values.
  warnings.warn("Unit cell dimensions not found. "
/shared/work/pnavarro/envs/deep_cartograph/lib/python3.10/site-packages/M

<Figure size 500x400 with 0 Axes>

<Figure size 500x400 with 0 Axes>

...

<Figure size 500x400 with 0 Axes>

<Figure size 500x400 with 0 Axes>

<Figure size 500x400 with 0 Axes>

<Figure size 500x400 with 0 Axes>

<Figure size 500x400 with 0 Axes>

In [20]:
# Show results for Deep TICA

for lt in lag_times:

    print(f"Lag time: {lt} ----------------------------------------------- #")

    # Test name
    test_name = f"lt_{lt}"
    
    # Show main results
    show_results(features, system_name, 'deep_tica', test_name)

Lag time: 1 ----------------------------------------------- #
Output folder: ../../../data/calpha_transitions/experiments/output/2olu_A-2olv_A-1/torsions/lt_1
Final model score: -1.9195E+0
Eigenvalue 1: 9.8996E-1
Eigenvalue 2: 9.6897E-1


Lag time: 2 ----------------------------------------------- #
Output folder: ../../../data/calpha_transitions/experiments/output/2olu_A-2olv_A-1/torsions/lt_2
Final model score: -9.5201E+1
Eigenvalue 1: 2.2902E+0
Eigenvalue 2: -2.8179E-1


Lag time: 4 ----------------------------------------------- #
Output folder: ../../../data/calpha_transitions/experiments/output/2olu_A-2olv_A-1/torsions/lt_4
Final model score: -3.2059E+2
Eigenvalue 1: 3.1755E+0
Eigenvalue 2: -1.3986E+0


Lag time: 8 ----------------------------------------------- #
Output folder: ../../../data/calpha_transitions/experiments/output/2olu_A-2olv_A-1/torsions/lt_8
Final model score: -1.8162E+0
Eigenvalue 1: 1.0153E+0
Eigenvalue 2: 8.7031E-1


Lag time: 16 ----------------------------------------------- #
Output folder: ../../../data/calpha_transitions/experiments/output/2olu_A-2olv_A-1/torsions/lt_16
Final model score: -1.8405E+0
Eigenvalue 1: 1.0926E+0
Eigenvalue 2: 8.0412E-1


- Lag time should be 1